In [1]:
%cd /content
!git clone -b magnet https://github.com/camenduru/audiocraft
%cd /content/audiocraft

!pip install -q av julius omegaconf einops num2words flashy torchmetrics sentencepiece
!pip install -q https://github.com/camenduru/wheels/releases/download/colab/xformers-0.0.20+1dc3d7a.d20240115-cp310-cp310-linux_x86_64.whl
!pip install -q .

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/facebook/audio-magnet-medium/resolve/main/state_dict.bin -d /content/audiocraft/audio-magnet-medium -o state_dict.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/facebook/audio-magnet-medium/resolve/main/compression_state_dict.bin -d /content/audiocraft/audio-magnet-medium -o compression_state_dict.bin

/content
Cloning into 'audiocraft'...
remote: Enumerating objects: 1258, done.
remote: Counting objects: 100% (546/546), done.
remote: Compressing objects: 100% (190/190), done.
remote: Total 1258 (delta 389), reused 356 (delta 356), pack-reused 712 (from 1)
Receiving objects: 100% (1258/1258), 1.73 MiB | 4.09 MiB/s, done.
Resolving deltas: 100% (715/715), done.
/content/audiocraft
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.4 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 8.5 MB/s eta 0:00:00
  Installing build dependencies ...

In [2]:
!pip show torch
!pip show transformers

Name: torch
Version: 2.1.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, audiocraft, demucs, dora_search, encodec, fastai, flashy, julius, openunmix, peft, sentence-transformers, timm, torchaudio, torchmetrics, torchvision, xformers
Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our co

In [3]:
!pip install torchvision==0.16.0
!pip install torchaudio==2.1.0
!pip install torch==2.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.0 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.20.1+cu121
    Uninstalling torchvision-0.20.1+cu121:
      Successfully uninstalled torchvision-0.20.1+cu121


In [4]:
%cd /content/audiocraft

from audiocraft.models import MAGNeT
model = MAGNeT.get_pretrained('/content/audiocraft/audio-magnet-medium')

model.set_generation_params(
    use_sampling=True,
    top_k=0,
    top_p=0.9,
    temperature=3.0,
    max_cfg_coef=10.0,
    min_cfg_coef=1.0,
    decoding_steps=[int(20 * model.lm.cfg.dataset.segment_duration // 10),  10, 10, 10],
    span_arrangement='nonoverlap'
)

from audiocraft.utils.notebook import display_audio

###### Text-to-music prompts - examples ######
text = "80s electronic track with melodic synthesizers, catchy beat and groovy bass"
# text = "80s electronic track with melodic synthesizers, catchy beat and groovy bass. 170 bpm"
# text = "Earthy tones, environmentally conscious, ukulele-infused, harmonic, breezy, easygoing, organic instrumentation, gentle grooves"
# text = "Funky groove with electric piano playing blue chords rhythmically"
# text = "Rock with saturated guitars, a heavy bass line and crazy drum break and fills."
# text = "A grand orchestral arrangement with thunderous percussion, epic brass fanfares, and soaring strings, creating a cinematic atmosphere fit for a heroic battle"

N_VARIATIONS = 3
descriptions = [text for _ in range(N_VARIATIONS)]

print(f"text prompt: {text}\n")
output = model.generate(descriptions=descriptions, progress=True, return_tokens=True)
display_audio(output[0], sample_rate=model.compression_model.sample_rate)

/content/audiocraft


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

text prompt: 80s electronic track with melodic synthesizers, catchy beat and groovy bass



In [5]:
%cd /content/audiocraft

from audiocraft.models import MAGNeT
model = MAGNeT.get_pretrained('/content/audiocraft/audio-magnet-medium')

model.set_generation_params(
    use_sampling=True,
    top_k=0,
    top_p=0.8,
    temperature=3.5,
    max_cfg_coef=20.0,
    min_cfg_coef=1.0,
    decoding_steps=[int(20 * model.lm.cfg.dataset.segment_duration // 10),  10, 10, 10],
    span_arrangement='nonoverlap'
)

from audiocraft.utils.notebook import display_audio

###### Text-to-audio prompts - examples ######
text = "Seagulls squawking as ocean waves crash while wind blows heavily into a microphone."
# text = "A toilet flushing as music is playing and a man is singing in the distance."

N_VARIATIONS = 3
descriptions = [text for _ in range(N_VARIATIONS)]

print(f"text prompt: {text}\n")
output = model.generate(descriptions=descriptions, progress=True, return_tokens=True)
display_audio(output[0], sample_rate=model.compression_model.sample_rate)

/content/audiocraft


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


text prompt: Seagulls squawking as ocean waves crash while wind blows heavily into a microphone.



In [6]:
%cd /content/audiocraft

from audiocraft.models import MAGNeT
model = MAGNeT.get_pretrained('/content/audiocraft/audio-magnet-medium')

model.set_generation_params(
    use_sampling=True,
    top_k=0,
    top_p=0.8,
    temperature=3.5,
    max_cfg_coef=20.0,
    min_cfg_coef=1.0,
    decoding_steps=[int(20 * model.lm.cfg.dataset.segment_duration // 10),  10, 10, 10],
    span_arrangement='nonoverlap'
)

from audiocraft.utils.notebook import display_audio

###### Text-to-audio prompts - examples ######
# text = "Seagulls squawking as ocean waves crash while wind blows heavily into a microphone."
text = "A toilet flushing as music is playing and a man is singing in the distance."

N_VARIATIONS = 3
descriptions = [text for _ in range(N_VARIATIONS)]

print(f"text prompt: {text}\n")
output = model.generate(descriptions=descriptions, progress=True, return_tokens=True)
display_audio(output[0], sample_rate=model.compression_model.sample_rate)

/content/audiocraft


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


text prompt: A toilet flushing as music is playing and a man is singing in the distance.

